## Part 1: Solving a Linear System Using the Inverse Matrix

### Core Concept
A system of $n$ linear equations with $n$ variables can be represented in matrix form as:
$$ Ax = b $$
where:
- **A** is an $n \times n$ square matrix of the coefficients.
- **x** is an $n \times 1$ column vector of the unknown variables.
- **b** is an $n \times 1$ column vector of the constant terms on the right-hand side.

### The Solution Method
If the matrix **A** is **non-singular**, it has a unique inverse, denoted as **A⁻¹**. A matrix is non-singular if and only if its determinant is non-zero (`det(A) ≠ 0`).

The solution vector **x** can be found by pre-multiplying both sides of the equation by **A⁻¹**:
$$ A^{-1}Ax = A^{-1}b $$
Since $A^{-1}A = I$ (the identity matrix), the equation simplifies to:
$$ x = A^{-1}b $$

### Limitations
While conceptually simple, this method has practical drawbacks:
1.  **Computational Cost:** Calculating the inverse of a matrix is a computationally intensive process, especially for large matrices.
2.  **Numerical Instability:** For matrices that are "ill-conditioned" (nearly singular), this method can lead to large numerical errors in the solution.

For these reasons, while it is a valid theoretical approach, it is often less preferred in practice compared to methods like Gaussian Elimination.

In [ ]:
import numpy as np

def get_result_by_inverse_matrix(A, b):
    """
    Solves the linear system Ax = b using the inverse matrix method.

    Args:
      A (np.ndarray): The n x n coefficient matrix.
      b (np.ndarray): The n x 1 results vector.

    Returns:
      (np.ndarray): The solution vector x, or None if the matrix is singular.
    """
    # Defensive programming: ensure the matrix is square
    rows, cols = A.shape
    if rows != cols:
        print("Error: Input matrix A must be square.")
        return None

    # --- Step 1: Check for Singularity ---
    # A unique solution exists only if the determinant is non-zero.
    determinant = np.linalg.det(A)
    print(f"Determinant of A: {determinant:.2f}")

    if np.isclose(determinant, 0): # Use np.isclose for floating-point comparison
        print("Matrix is singular or nearly singular. No unique solution exists.")
        return None

    # --- Step 2: Calculate the Inverse ---
    # If the check passes, we can safely compute the inverse.
    print("Matrix is non-singular. Calculating inverse...")
    A_inv = np.linalg.inv(A)

    # --- Step 3: Find the Solution ---
    # Multiply the inverse by the b vector to get x.
    x = A_inv @ b # Using the @ operator for matrix-vector multiplication

    return x

In [ ]:
# --- Test for Inverse Matrix Method ---
# The linear system is:
# 1x + 2y + 1z = 0
# 1x - 2y + 2z = 4
# 2x + 12y - 2z = 4

data_A = np.array([[1, 2, 1], [1, -2, 2], [2, 12, -2]])
data_b = np.array([0, 4, 4])

solution_x = get_result_by_inverse_matrix(data_A, data_b)

if solution_x is not None:
    print(f"\nThe solution vector is: {solution_x}")

    # --- Verification ---
    # The ultimate test: does A @ x produce our original b?
    b_recreated = data_A @ solution_x
    print(f"Verification (A @ x): {b_recreated}")
    print(f"Original b vector:    {data_b}")
    np.testing.assert_array_almost_equal(b_recreated, data_b)
    print("\nVerification successful!")

Determinant of A: 8.00
Matrix is non-singular. Calculating inverse...

The solution vector is: [11.  -2.5 -6. ]
Verification (A @ x): [0. 4. 4.]
Original b vector:    [0 4 4]

Verification successful!


## Part 2: Solving a Linear System Using Gaussian Elimination

### Core Concept
Gaussian Elimination is a systematic algorithm for solving systems of linear equations by transforming the original system `Ax = b` into an equivalent, but much simpler, **upper triangular system** `Ux = y`. This new system can then be easily solved without computing a matrix inverse.

The transformation is achieved using **elementary row operations**:
1.  Adding a multiple of one row to another.
2.  Swapping two rows (Pivoting - an important extension, not required by this lab's core task).
3.  Multiplying a row by a non-zero constant.

These operations are performed on an **augmented matrix** `[A|b]`, which is the `A` matrix with the `b` vector appended as an extra column. This ensures that the operations are applied to both `A` and `b` simultaneously, preserving the solution `x`.

### The Algorithm: A Two-Step Process

**Step 1: Forward Elimination**

This is the main phase where we introduce zeros below the main diagonal to create the upper triangular matrix `U`.

-   **Goal:** For each column `k` from `0` to `n-2`, we want to make all elements below the diagonal element `A[k,k]` equal to zero.
-   **Process for column `k`:**
    1.  The row `k` is the **pivot row**, and `A[k,k]` is the **pivot element**.
    2.  For each row `i` below the pivot row (i.e., `i > k`):
        a. Calculate a **multiplier**: `m = A[i, k] / A[k, k]`.
        b. Update the entire row `i` with the operation: `Row_i = Row_i - m * Row_k`.
-   **Result:** After iterating through all columns, the original augmented matrix `[A|b]` is transformed into `[U|y]`, where `U` is upper triangular.

**Step 2: Backward Substitution**

Now that the system is in the simple form `Ux = y`, we can solve for the variables starting from the bottom up.

-   **Goal:** Find the values of $x_n, x_{n-1}, \dots, x_1$.
-   **Process:**
    1.  The last equation in the system is `U[n-1, n-1] * x[n-1] = y[n-1]`. We can solve this directly for the last variable:
        $$ x_{n-1} = \frac{y_{n-1}}{U_{n-1, n-1}} $$
    2.  The second-to-last equation is `U[n-2, n-2] * x[n-2] + U[n-2, n-1] * x[n-1] = y[n-2]`. Since we now know $x_{n-1}$, we can plug it in and solve for $x_{n-2}$.
    3.  This process is repeated, working backwards up the matrix. The general formula is:
        $$ x_j = \frac{1}{U_{j,j}} \left( y_j - \sum_{k=j+1}^{n-1} U_{j,k} x_k \right) $$
        for $j = n-2, \dots, 0$.

This two-step process is more numerically stable and computationally efficient for large systems than the inverse matrix method.

In [ ]:
import numpy as np
import sys # Used for sys.exit in case of pivoting issues

def get_result_gaussian_elimination(n, A):
    """
    Solves the linear system Ax = b using Gaussian Elimination with
    Backward Substitution.

    Args:
      n (int): The number of equations/variables (size of the square matrix A).
      A (np.ndarray): The n x (n+1) augmented matrix [A|b].

    Returns:
      (np.ndarray): The solution vector x.
    """
    # Create a floating-point copy to avoid modifying the original array
    # and to ensure floating point division works as expected.
    A = A.astype(np.float64)

    #---------------------------------------------------------------------
    # Step 1: Forward Elimination
    #---------------------------------------------------------------------
    # Iterate through the columns, making each the pivot column
    for i in range(n):

        # --- Pivoting Check ---
        # If the pivot element A[i,i] is zero, a unique solution might not exist
        # without row swapping (pivoting). For this lab's scope,
        # we'll exit if this happens.
        if A[i, i] == 0.0:
            sys.exit('Divide by zero detected! Pivoting would be required.')

        # Iterate through the rows below the current pivot row
        for j in range(i + 1, n):
            # Calculate the multiplier
            multiplier = A[j, i] / A[i, i]

            # Apply the row operation: Row_j = Row_j - multiplier * Row_i
            # This is applied to the entire augmented row
            A[j] = A[j] - multiplier * A[i]

    print("--- Upper Triangular Matrix ---")
    print(np.round(A, 3)) # Round for cleaner display

    #---------------------------------------------------------------------
    # Step 2: Backward Substitution
    #---------------------------------------------------------------------
    # Create an empty vector to store our solution x
    x = np.zeros(n)

    # First, solve for the last variable, x[n-1], directly.
    # The last row of the augmented matrix represents U[n-1,n-1]*x[n-1] = y[n-1]
    x[n-1] = A[n-1, n] / A[n-1, n-1]

    # Now, loop backwards from the second-to-last row (n-2) up to the first row (0)
    for i in range(n - 2, -1, -1):
        # This part implements the formula:
        # x[i] = ( y[i] - sum(U[i,k]*x[k] for k from i+1 to n-1) ) / U[i,i]

        # The sum part can be done efficiently with a dot product
        sum_term = np.dot(A[i, i+1:n], x[i+1:n])

        # The y[i] term is now A[i,n] in our modified augmented matrix
        x[i] = (A[i, n] - sum_term) / A[i, i]

    return x

In [ ]:
# --- Test for Gaussian Elimination Method ---
# The same linear system
data_n = 3
data_A_augmented = np.array([[1, 2, 1, 0],
                             [1, -2, 2, 4],
                             [2, 12, -2, 4]])

solution_x_gauss = get_result_gaussian_elimination(data_n, data_A_augmented)

print(f"\nThe solution vector is: {solution_x_gauss}")

# --- Verification ---
expected_solution = np.array([11., -2.5, -6.])
np.testing.assert_array_almost_equal(solution_x_gauss, expected_solution)
print("\nVerification successful!")

--- Upper Triangular Matrix ---
[[ 1.  2.  1.  0.]
 [ 0. -4.  1.  4.]
 [ 0.  0. -2. 12.]]

The solution vector is: [11.  -2.5 -6. ]

Verification successful!


## Daily Evaluation - 4 Marks

1.  Given the matrix `A = [[2, -1, 0], [-1, 2, -1], [0, -1, 2]]`, use NumPy to explicitly calculate `A_inv = A⁻¹` and then verify that `A @ A_inv` results in the identity matrix `I`.

2.  Given the matrix `B = [[1, 2, 3, 4], [2, 4, 6, 8], [3, 6, 9, 12], [4, 8, 12, 16]]`, calculate its determinant using NumPy. What does the result of the determinant tell you about this matrix and its suitability for solving a linear system for a unique solution?

In [ ]:
import numpy as np

# --- Part 1 ---
print("--- Part 1: Verifying the Identity ---")
A = np.array([[2, -1, 0],
              [-1, 2, -1],
              [0, -1, 2]])

# Calculate the inverse
A_inv = np.linalg.inv(A)
print("Matrix A_inv:\n", A_inv)

# Verify A @ A_inv = I
identity = A @ A_inv
# We round the result to handle tiny floating point errors
identity_rounded = np.round(identity, decimals=5)

print("\nThe product A @ A_inv is:\n", identity_rounded)
print("\nThis is the 3x3 Identity Matrix, as expected.")

print("\n" + "="*50 + "\n")

# --- Part 2 ---
print("--- Part 2: Checking the Determinant ---")
B = np.array([[1, 2, 3, 4],
              [2, 4, 6, 8],
              [3, 6, 9, 12],
              [4, 8, 12, 16]])

# Calculate the determinant
det_B = np.linalg.det(B)
print(f"The determinant of matrix B is: {det_B}")

print("\n--- Implications ---")
print("Since the determinant is 0, the matrix B is singular.")
print("This means that B does not have an inverse, and the linear system Bx=b does not have a unique solution.")
print("The system will either have NO solution or INFINITELY MANY solutions, depending on the vector b.")

--- Part 1: Verifying the Identity ---
Matrix A_inv:
 [[0.75 0.5  0.25]
 [0.5  1.   0.5 ]
 [0.25 0.5  0.75]]

The product A @ A_inv is:
 [[ 1.  0.  0.]
 [ 0.  1. -0.]
 [-0. -0.  1.]]

This is the 3x3 Identity Matrix, as expected.


--- Part 2: Checking the Determinant ---
The determinant of matrix B is: 0.0

--- Implications ---
Since the determinant is 0, the matrix B is singular.
This means that B does not have an inverse, and the linear system Bx=b does not have a unique solution.
The system will either have NO solution or INFINITELY MANY solutions, depending on the vector b.
